# Django Workshop

Welcome to this notebook-workshop. 

Here you will be guided on how to use the Django ORM.

There are some examples and some exercise that you must complete.

If you have run `make init` then your database will be populated with some playing data.

Listen to the tutors

Remember to press **SHIFT + ENTER** to run the notebook line

Let's begin...

# https://github.com/Woile/django-orm-workshop

![humanresources](./hrtables.gif "Tables")

## Import the things we are gonna use

In [1]:
from tabulate import tabulate
from django.db.models import Max, Min, Avg, Sum, Count
from humanresources.models import (
    Regions,
    Countries,
    Locations,
    Departments,
    Jobs,
    Employees
)


def simple_table(queryset, *fields):
    """Usage:
    simple_table(jobs, 'job_title')
    """
    v = queryset.values_list(*fields)
    print(tabulate(v, headers=fields))

# Basic Usage

In order to make queries, you must always access the manager "objects". Example:

`Jobs.objects.all()`

## Creation

Simple as running `MyModel.objects.create` with the right parameters


Let's see an example of a `Regions` creation

In [25]:
region = Regions.objects.create(region_name="moon")

In [26]:
country = Countries.objects.create(country_iso_code="WK", country_name="Wakanda", region=region)
country

<Countries: (WK) Wakanda>

Now it's your turn.

**Exercise 1**

Create a Job. Go to `demo/humanresources/models.py` to see the fields

Modify the code in the next line to include the missing fields

In [ ]:
job = Jobs.objects.create()  # add fields inside create

## Update

Simple as running `MyModel.objects.update` with the right parameters

Let's update the region created above.


In [33]:
Countries.objects.filter(country_iso_code="WK").update(region=Regions.objects.get(region_name="Middle East and Africa"))


2

## Delete

Simple as running `MyModel.objects.delete` with the right parameters

Let's delete the region created above.


In [29]:
Regions.objects.filter(region_name="moon").delete()

(1, {'humanresources.Regions': 1})

# Filtering

Now the fun part starts, this would be doing a normal **SQL SELECT**.

Usually you'll have to do some lookups while filtering.
In order to do so, you must use double underscore (`__`) after the field name.

## Lookups

These are some of the lookups we are gonna use:

| Lookup | Description |
| :--- | ---: |
| [icontains](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#icontains) | insensitive contain |
| [in](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#in) | is in a list.  |
| [gt](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#gt) | greater than |
| [gte](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#gte) | greater than equal |
| [lt](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#lt) | lower than |
| [lte](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#lte) | lower than equal  |
| [range](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#range) | is in the given range, works with dates also. |
| [year](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#year) | select the year from dates |
| [isnull](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#isnull) | is the field row is null. |


Examples: 

`Regions.objects.filter(region_name__icontains='ar')`

`Regions.objects.filter(id__in=[1, 2, 3])`

Let's retrieve some information from our data

**Exercise 2**

Filter the Jobs with minimum salary greater than 10000.

In [31]:
jobs = Jobs.objects.filter(min_salary__gt=10000)

simple_table(jobs, 'min_salary', 'job_title')
print(jobs.query)

  min_salary  job_title
------------  -----------------------------
       20000  President
       15000  Administration Vice President
SELECT "humanresources_jobs"."id", "humanresources_jobs"."job_code", "humanresources_jobs"."job_title", "humanresources_jobs"."min_salary", "humanresources_jobs"."max_salary" FROM "humanresources_jobs" WHERE "humanresources_jobs"."min_salary" > 10000.0


**Exercise 3**

Display the number of employees hired in 

**hint**: Fill the missing lookup

In [ ]:
Employees.objects.filter(hire_date=1999).count()

**Exercise 4**

Display the number of employees hired between `1999` and `2000`

**hint**: Fill the missing lookups

In [15]:
years_range = (1999, 2000)
Employees.objects.filter(hire_date__year__range=years_range).count()

29

**Exercise 5**

Display the department names without a manager

**hint**: Fill the missing lookup

In [16]:
departments = Departments.objects.filter(manager_id__isnull=True)

simple_table(departments, 'department_name')

department_name
--------------------
Treasury
Corporate Tax
Control And Credit
Shareholder Services
Benefits
Manufacturing
Construction
Contracting
Operations
IT Support
NOC
IT Helpdesk
Government Sales
Retail Sales
Recruiting
Payroll


## Aggregations

[Documentation](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#aggregation-functions)

[Cheat Sheet](https://docs.djangoproject.com/en/2.1/topics/db/aggregation/#cheat-sheet)

Now you'll see how to retrieve valuable information from your queries

**Exercise 6**

Display the max salary earned in the company

In [9]:
Employees.objects.aggregate(Max('salary')) 

{'salary__max': 24000.0}

**Exercise 7**

Display the number of employees having the max salary

**hint** Fill the missing aggregate

In [17]:
Employees.objects.filter(salary=Employees.objects.aggregate(Max('salary'))['salary__max']).count()  

1

**Exercise 8**

Display the min salary earned in the company

**hint** Fill the missing aggregate

In [18]:
Employees.objects.aggregate(Min('salary')) 

{'salary__min': 2100.0}

**Exercise 9**

Display the avg salary earned in the company

**hint** Fill the missing aggregate

In [19]:
Employees.objects.aggregate(Avg('salary')) 

{'salary__avg': 6480.95238095238}

**Exercise 10**

Display the number of employees having the max salary

**hint** Fill the missing functions

In [20]:
qs = Employees.objects.aggregate(Max('salary'), Avg('salary'))  
Employees.objects.filter(salary__range=(qs['salary__avg'], qs['salary__max'])).count() 

50

**Example**

Display the number of employees per department name

In [12]:
department_counts = Employees.objects.values('department__department_name').annotate(dep_count=Count('department__department_name'))  

print(department_counts.query)

SELECT "humanresources_departments"."department_name", COUNT("humanresources_departments"."department_name") AS "dep_count" FROM "humanresources_employees" LEFT OUTER JOIN "humanresources_departments" ON ("humanresources_employees"."department_id" = "humanresources_departments"."id") GROUP BY "humanresources_departments"."department_name"


**Exercise 11**

Display the department name with 2 employees

In [21]:
department_counts.filter(dep_count=2).values('department__department_name')

<QuerySet [{'department__department_name': 'Accounting'}, {'department__department_name': 'Marketing'}]>

## Optimizations

**Remember: pre optimization is the root of all evil**

If you see your queries are behaving slowly, there are some simple tweaks you can do to make them faster.

### select_related

[Documentation](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#select-related)

Will include `JOINS`

In [25]:
print(Employees.objects.all().query)

SELECT "humanresources_employees"."id", "humanresources_employees"."first_name", "humanresources_employees"."last_name", "humanresources_employees"."email", "humanresources_employees"."hire_date", "humanresources_employees"."job_id", "humanresources_employees"."salary", "humanresources_employees"."manager_id", "humanresources_employees"."department_id", "humanresources_employees"."commission_pct", "humanresources_employees"."phone_number" FROM "humanresources_employees"


In [24]:
print(Employees.objects.select_related('department').all().query)

SELECT "humanresources_employees"."id", "humanresources_employees"."first_name", "humanresources_employees"."last_name", "humanresources_employees"."email", "humanresources_employees"."hire_date", "humanresources_employees"."job_id", "humanresources_employees"."salary", "humanresources_employees"."manager_id", "humanresources_employees"."department_id", "humanresources_employees"."commission_pct", "humanresources_employees"."phone_number", "humanresources_departments"."id", "humanresources_departments"."department_name", "humanresources_departments"."manager_id", "humanresources_departments"."location_id" FROM "humanresources_employees" LEFT OUTER JOIN "humanresources_departments" ON ("humanresources_employees"."department_id" = "humanresources_departments"."id")


In [3]:
print(Employees.objects.select_related('department').filter(department__id__isnull=False).all().query)

SELECT "humanresources_employees"."id", "humanresources_employees"."first_name", "humanresources_employees"."last_name", "humanresources_employees"."email", "humanresources_employees"."hire_date", "humanresources_employees"."job_id", "humanresources_employees"."salary", "humanresources_employees"."manager_id", "humanresources_employees"."department_id", "humanresources_employees"."commission_pct", "humanresources_employees"."phone_number", "humanresources_departments"."id", "humanresources_departments"."department_name", "humanresources_departments"."manager_id", "humanresources_departments"."location_id" FROM "humanresources_employees" INNER JOIN "humanresources_departments" ON ("humanresources_employees"."department_id" = "humanresources_departments"."id") WHERE "humanresources_employees"."department_id" IS NOT NULL


**Exercise 12**

Display the employees  `first_name`, `last_name` and its `department_name` WITH a department assigned

**hint**: you'll have to use relationships and lookups here. [Docs](https://docs.djangoproject.com/en/2.1/topics/db/queries/#lookups-that-span-relationships)

In [24]:
employees = Employees.objects.filter(department__id__isnull=False)

simple_table(employees, 'first_name', 'last_name', 'department__department_name')

first_name    last_name    department__department_name
------------  -----------  -----------------------------
Steven        King         Executive
Neena         Kochhar      Executive
Lex           De Haan      Executive
Alexander     Hunold       IT
Bruce         Ernst        IT
David         Austin       IT
Valli         Pataballa    IT
Diana         Lorentz      IT
Nancy         Greenberg    Finance
Daniel        Faviet       Finance
John          Chen         Finance
Ismael        Sciarra      Finance
Jose Manuel   Urman        Finance
Luis          Popp         Finance
Den           Raphaely     Purchasing
Alexander     Khoo         Purchasing
Shelli        Baida        Purchasing
Sigal         Tobias       Purchasing
Guy           Himuro       Purchasing
Karen         Colmenares   Purchasing
Matthew       Weiss        Shipping
Adam          Fripp        Shipping
Payam         Kaufling     Shipping
Shanta        Vollman      Shipping
Kevin         Mourgos      Shipping
Julia    

**Exercise 13**

Display the first and last name, department, city, and state province for each employee

**hint**: you'll have to use relationships and lookups here. [Docs](https://docs.djangoproject.com/en/2.1/topics/db/queries/#lookups-that-span-relationships)

In [ ]:
employees = Employees.objects.values(...)  

**Exercise 14**

Display those employees who contain a letter z to their first name and also display their last name, department, city, and state province.

**hint**: you'll have to use relationships and lookups here. [Docs](https://docs.djangoproject.com/en/2.1/topics/db/queries/#lookups-that-span-relationships)

In [ ]:
Employees.objects.values(...).filter(...)

## prefect_related

[Documentation](https://docs.djangoproject.com/en/2.1/ref/models/querysets/#prefetch-related)

Helps with many to many queries